In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd

In [53]:
# Loading datasets

training_labels = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_train/train_labels.csv')
training_input = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_train/train_images.csv',
                                header = None)

testing_labels = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_test/test_labels.csv')
testing_input = pd.read_csv('/home/sesso/Documents/Study/pytorch_learn/volcanoes_venus/venus_data/archive/volcanoes_test/test_images.csv',
                                header = None)           

# Creating tensor dataset

target_col = 'Type'

# Make nan in targe col = 0 
training_labels[target_col] = training_labels[target_col].fillna(0)
testing_labels[target_col] = testing_labels[target_col].fillna(0)

# use 30% of training data for validation
train_set_size = int(len(training_input) * 0.7)
valid_set_size = len(training_input) - train_set_size
batch_size = 64

## Convert to numpy array

training_labels = training_labels[target_col].to_numpy()
training_input = training_input.to_numpy()

testing_labels = testing_labels[target_col].to_numpy()
testing_input = testing_input.to_numpy()

## Make tensors of them

training_inputs = torch.tensor(training_input, dtype=torch.float32)
training_targets = torch.tensor(training_labels, dtype=torch.float32)

testing_inputs = torch.tensor(testing_input, dtype=torch.float32)
testing_targets = torch.tensor(testing_labels, dtype=torch.float32)

## Tensor datasets

dtrain = TensorDataset(training_inputs, training_targets)
dtest = TensorDataset(testing_inputs, testing_targets)

## Split train and validation set

### split the train set into two
seed = torch.Generator().manual_seed(42)
train_set, valid_set = random_split(dtrain, [train_set_size, valid_set_size], generator=seed)

### Batch loader

train_dl = DataLoader(train_set, batch_size = batch_size, shuffle=True) 
val_dl = DataLoader(valid_set, batch_size = batch_size, shuffle=True) 
test_dl = DataLoader(dtest, batch_size =batch_size)



In [3]:
for X, y in test_dl:
    print(f"Shape of X [N, C]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C]: torch.Size([64, 12100])
Shape of y: torch.Size([64]) torch.int32


In [84]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # self.flatten = nn.Flatten()
        self.seq = nn.Sequential(
            nn.Linear(110*110, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Linear(10,5),
            nn.LogSoftmax(-1)
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.seq(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (seq): Sequential(
    (0): Linear(in_features=12100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=5, bias=True)
    (6): LogSoftmax(dim=-1)
  )
)


In [82]:
# loss_fn = nn.CrossEntropyLoss()
from tkinter import Variable


loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader): # CHECK
        print(0)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        _, pred = torch.max(pred, 1)
        
        # pred = pred[:-1]
        # pred = pred.argmax(1)

        # pred = torch.tensor(pred, dtype=torch.float32)
        print('pred')
        print(pred)
        
        print('y')
        print(y)
        loss = loss_fn(pred, y)
        # loss = Variable(loss, requires_grad = TRUE)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [80]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [85]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dl, model, loss_fn, optimizer)
    test(test_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
0
pred
tensor([3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0,
        3, 3, 4, 3, 3, 0, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 0, 3, 0, 0, 4, 3, 3,
        3, 3, 4, 0, 3, 0, 3, 3, 4, 3, 3, 3, 3, 4, 0, 3])
y
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 3., 4., 0., 0., 4., 4., 0., 0., 2., 4., 0.,
        0., 0., 0., 0., 0., 4., 0., 0., 0., 4., 4., 3., 0., 3., 0., 3., 4., 0.,
        0., 3., 0., 4., 4., 0., 0., 0., 0., 0.])


RuntimeError: "log_softmax_lastdim_kernel_impl" not implemented for 'Long'